ਆਟੋਜਨ ਟੂਲ ਵਰਤੋਂ ਉਦਾਹਰਨ


## ਲੋੜੀਂਦੇ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰੋ


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## ਕਲਾਇੰਟ ਬਣਾਉਣਾ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ LLM ਤੱਕ ਪਹੁੰਚ ਲਈ।

`model` ਨੂੰ `gpt-4o-mini` ਵਜੋਂ ਪਰਿਭਾਸ਼ਿਤ ਕੀਤਾ ਗਿਆ ਹੈ। GitHub Models ਮਾਰਕੀਟਪਲੇਸ ਵਿੱਚ ਉਪਲਬਧ ਕਿਸੇ ਹੋਰ ਮਾਡਲ ਨਾਲ ਮਾਡਲ ਬਦਲਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੋ ਤਾਂ ਜੋ ਵੱਖ-ਵੱਖ ਨਤੀਜੇ ਵੇਖੇ ਜਾ ਸਕਣ।

ਇੱਕ ਤੇਜ਼ ਟੈਸਟ ਵਜੋਂ, ਅਸੀਂ ਸਿਰਫ ਇੱਕ ਸਧਾਰਨ ਪ੍ਰੰਪਟ ਚਲਾਵਾਂਗੇ - `ਫਰਾਂਸ ਦੀ ਰਾਜਧਾਨੀ ਕੀ ਹੈ`।


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ਫੰਕਸ਼ਨਾਂ ਦੀ ਪਰਿਭਾਸ਼ਾ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਏਜੰਟ ਨੂੰ ਇੱਕ ਟੂਲ ਤੱਕ ਪਹੁੰਚ ਦੇਵਾਂਗੇ ਜੋ ਇੱਕ ਫੰਕਸ਼ਨ ਹੈ ਜਿਸ ਵਿੱਚ ਉਪਲਬਧ ਛੁੱਟੀਆਂ ਦੇ ਗੰਤਵਿਆਂ ਦੀ ਸੂਚੀ ਅਤੇ ਉਨ੍ਹਾਂ ਦੀ ਉਪਲਬਧਤਾ ਹੈ।

ਤੁਸੀਂ ਇਹ ਸੋਚ ਸਕਦੇ ਹੋ ਕਿ ਇਹ ਉਹ ਸਥਿਤੀ ਹੋਵੇਗੀ ਜਿੱਥੇ ਇੱਕ ਟ੍ਰੈਵਲ ਏਜੰਟ, ਉਦਾਹਰਣ ਵਜੋਂ, ਇੱਕ ਟ੍ਰੈਵਲ ਡੇਟਾਬੇਸ ਤੱਕ ਪਹੁੰਚ ਰੱਖ ਸਕਦਾ ਹੈ।

ਜਦੋਂ ਤੁਸੀਂ ਇਸ ਨਮੂਨੇ ਵਿੱਚੋਂ ਗੁਜ਼ਰਦੇ ਹੋ, ਤਾਂ ਨਵੀਆਂ ਫੰਕਸ਼ਨਾਂ ਅਤੇ ਟੂਲਾਂ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਨ ਲਈ ਖੁਦ ਨੂੰ ਆਜ਼ਾਦ ਮਹਿਸੂਸ ਕਰੋ, ਜਿਨ੍ਹਾਂ ਨੂੰ ਏਜੰਟ ਕਾਲ ਕਰ ਸਕਦਾ ਹੈ।


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## ਫੰਕਸ਼ਨ ਟੂਲ ਦੀ ਪਰਿਭਾਸ਼ਾ  
ਏਜੰਟ ਨੂੰ `vacation_destinations` ਨੂੰ ਇੱਕ `FunctionTool` ਵਜੋਂ ਵਰਤਣ ਲਈ, ਸਾਨੂੰ ਇਸਨੂੰ ਇੱਕ ਟੂਲ ਵਜੋਂ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨਾ ਪਵੇਗਾ।  

ਅਸੀਂ ਟੂਲ ਦੀ ਇੱਕ ਵਰਣਨਾ ਵੀ ਪ੍ਰਦਾਨ ਕਰਾਂਗੇ ਜੋ ਏਜੰਟ ਨੂੰ ਇਹ ਪਛਾਣ ਕਰਨ ਵਿੱਚ ਮਦਦ ਕਰਦੀ ਹੈ ਕਿ ਇਸ ਟੂਲ ਨੂੰ ਉਪਭੋਗਤਾ ਦੁਆਰਾ ਕੀਤੇ ਗਏ ਕੰਮ ਦੇ ਸੰਦਰਭ ਵਿੱਚ ਕਿਵੇਂ ਵਰਤਿਆ ਜਾ ਸਕਦਾ ਹੈ।  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ਏਜੰਟ ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਨਾ

ਹੁਣ ਅਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਕੋਡ ਵਿੱਚ ਏਜੰਟ ਬਣਾਉਣ ਜਾ ਰਹੇ ਹਾਂ। ਅਸੀਂ `system_message` ਨੂੰ ਪਰਿਭਾਸ਼ਿਤ ਕਰਦੇ ਹਾਂ ਤਾਂ ਜੋ ਏਜੰਟ ਨੂੰ ਯੂਜ਼ਰਾਂ ਦੀ ਮਦਦ ਕਰਨ ਲਈ ਹਦਾਇਤਾਂ ਦਿੱਤੀਆਂ ਜਾ ਸਕਣ ਕਿ ਉਹ ਛੁੱਟੀਆਂ ਦੇ ਗੰਤਵਿਆਂ ਨੂੰ ਕਿਵੇਂ ਲੱਭ ਸਕਦੇ ਹਨ।

ਅਸੀਂ `reflect_on_tool_use` ਪੈਰਾਮੀਟਰ ਨੂੰ true 'ਤੇ ਸੈਟ ਕਰਦੇ ਹਾਂ। ਇਹ LLM ਨੂੰ ਟੂਲ ਕਾਲ ਦੇ ਜਵਾਬ ਨੂੰ ਲੈਣ ਅਤੇ ਕੁਦਰਤੀ ਭਾਸ਼ਾ ਵਿੱਚ ਜਵਾਬ ਭੇਜਣ ਦੀ ਆਗਿਆ ਦਿੰਦਾ ਹੈ।

ਤੁਸੀਂ ਇਸ ਪੈਰਾਮੀਟਰ ਨੂੰ false 'ਤੇ ਸੈਟ ਕਰ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਅੰਤਰ ਵੇਖ ਸਕੋ।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ਏਜੰਟ ਚਲਾਉਣਾ

ਹੁਣ ਅਸੀਂ ਸ਼ੁਰੂਆਤੀ ਯੂਜ਼ਰ ਸੁਨੇਹੇ ਨਾਲ ਏਜੰਟ ਚਲਾ ਸਕਦੇ ਹਾਂ, ਜਿਸ ਵਿੱਚ ਟੋਕੀਓ ਦੀ ਯਾਤਰਾ ਕਰਨ ਦੀ ਗੱਲ ਕੀਤੀ ਗਈ ਹੈ।

ਤੁਸੀਂ ਇਸ ਸ਼ਹਿਰ ਦੇ ਗੰਤੀਸਥਾਨ ਨੂੰ ਬਦਲ ਸਕਦੇ ਹੋ ਤਾਂ ਜੋ ਦੇਖ ਸਕੋ ਕਿ ਸ਼ਹਿਰ ਦੀ ਉਪਲਬਧਤਾ ਲਈ ਏਜੰਟ ਕਿਵੇਂ ਪ੍ਰਤੀਕਿਰਿਆ ਕਰਦਾ ਹੈ।


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਣਭਵਤਾਵਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੇ ਪ੍ਰਯੋਗ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਮੀਆਂ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆਵਾਂ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
